In [29]:
import pandas as pd

df = pd.read_csv("../data/violations.csv")
df.head(10)

,case_no,status_dttm,status,code,value,description,violation_stno,violation_sthigh,violation_street,violation_suffix,...,ward,contact_addr1,contact_addr2,contact_city,contact_state,contact_zip,sam_id,latitude,longitude,location
0,HVIOL-662883,2023-02-22 16:34:56,Open,CMR410.482 A,NaN,Smoke Detectors & Carbon Monoxide Alarms - Own...,116,NaN,Hutchings,ST,...,12,116 Hutchings St #1,NaN,Dorchester,MA,02121,256986,42.311610,-71.092340,"(42.31161000001751, -71.09234000120948)"
1,HVIOL-662883,2023-02-22 16:34:56,Open,CMR410.500,NaN,Owners Responsibility to Maintain Structural E...,116,NaN,Hutchings,ST,...,12,116 Hutchings St #1,NaN,Dorchester,MA,02121,256986,42.311610,-71.092340,"(42.31161000001751, -71.09234000120948)"
2,HVIOL-662883,2023-02-22 16:34:56,Open,CMR410.482 A,NaN,Smoke Detectors & Carbon Monoxide Alarms - Own...,116,NaN,Hutchings,ST,...,12,116 Hutchings St #1,NaN,Dorchester,MA,02121,256986,42.311610,-71.092340,"(42.31161000001751, -71.09234000120948)"
3,HVIOL-662883,2023-02-22 16:34:56,Open,CMR410.482 A,NaN,Smoke Detectors & Carbon Monoxide Alarms - Own...,116,NaN,Hutchings,ST,...,12,116 Hutchings St #1,NaN,Dorchester,MA,02121,256986,42.311610,-71.092340,"(42.31161000001751, -71.09234000120948)"
4,HVIOL-662883,2023-02-22 16:34:56,Open,CMR410.500,NaN,Owners Responsibility to Maintain Structural E...,116,NaN,Hutchings,ST,...,12,116 Hutchings St #1,NaN,Dorchester,MA,02121,256986,42.311610,-71.092340,"(42.31161000001751, -71.09234000120948)"
5,HVIOL-662883,2023-02-22 16:34:56,Open,CMR410.482 A,NaN,Smoke Detectors & Carbon Monoxide Alarms - Own...,116,NaN,Hutchings,ST,...,12,116 Hutchings St #1,NaN,Dorchester,MA,02121,256986,42.311610,-71.092340,"(42.31161000001751, -71.09234000120948)"
6,HVIOL-662682,2023-02-21 15:29:54,Open,CMR410.351 B,NaN,Owners Installation/Maintenance Responsibility...,974,NaN,Saratoga,ST,...,01,60 Border St,NaN,East Boston,MA,02128,306474,42.387110,-71.007620,"(42.38711000004081, -71.007620001253)"
7,HVIOL-662681,2023-02-21 15:29:32,Open,CMR410.482 A,NaN,Smoke Detectors & Carbon Monoxide Alarms - Own...,734,NaN,Cummins,HWY,...,18,104 Thatcher St.,NaN,Milton,MA,02186,221206,42.270911,-71.101907,"(42.27091100003018, -71.10190700119084)"
8,V650006,2023-02-21 15:18:06,Open,116,NaN,Unsafe Structures,690,710,American Legion,HWY,...,18,34 Essex St,NaN,Andover,MA,01810,3179,42.283334,-71.112034,"(42.28333399981036, -71.1120340009656)"
9,V619158,2023-02-21 15:13:54,Open,102.8,NaN,Maintenance,690,710,American Legion,HWY,...,18,34 ESSEX STREET,NaN,ANDOVER,MA,01810,3179,42.283334,-71.112034,"(42.28333399981036, -71.1120340009656)"


In [5]:
# Hard-Coded Dictionary of hazardous conditions + violation codes
violations = {"Hot Water": "410.190", "Potable Water": "410.180", "Heating Facilities Required": "410.200", "Temperature Requirements": "410.201",
              "Asbestos Material": "410.353", "Use of Lead Paint Prohibited": "410.502"}

#Convert to lower case and remove whitespace so that we don't get duplicates
df['contact_addr1'] = df['contact_addr1'].apply(lambda x : str(x).lower())
df['contact_city'] = df['contact_city'].apply(lambda x : str(x).lower().replace(' ', ''))

In [49]:
import pandas as pd
import numpy as np
from folium import plugins, Map
from folium.plugins import HeatMap
from folium import Marker
from folium import Circle

# create a scatter map for all violation types
base_map_violations = Map(location=[42.357851, -71.124111])

# Hot Water: red; Potable Water: blue; Heating: green; Temperature: yellow; Asbestos: purple; Lead: cyan
colors = ['red', 'blue', 'green', 'yellow', 'purple', 'cyan']

# Go through each of the hazardous conditions violations
count = 0
for name, code in violations.items():
    # Look at only this violation (code name in the dataset have other values beyond the code # so had to use contains)
    mask = df['code'].str.contains(code, case=False, na=False)
    df_code =  df[mask]
    lat_list = df_code.latitude.to_list()
    long_list = df_code.longitude.to_list()
    # for each location, generate a violation point
    for i in range(len(lat_list)):
        Circle([lat_list[i], long_list[i]], radius = 10, popup = name, color = colors[count], fill = True).add_to(base_map_violations)    
        base_map_violations.save('violations.html')
    count += 1


In [81]:
# test
code = list(violations.values())[0]
mask = df['code'].str.contains(code, case=False, na=False)
df_code = df[mask]
df_code
# count the violation
count = df_code['contact_city'].value_counts()
count

# df_violation = pd.DataFrame(count)
# df_violation

# add 0-N index
# df_violation = df_violation.reset_index()
# change the column title
# df_violation.columns = ['Location', 'Counts']

# create heat map base
# base_map_heat = Map(location=[42.357851, -71.124111])

# df['latitude'] = df['latitude'].astype(float)
# df['longitude'] = df['longitude'].astype(float)
# df['price'] = df['price'].astype(float)
# mean = df.groupby(['latitude', 'longitude'], as_index=False)['price'].mean()
# HeatMap(mean).add_to(base_map)
# base_map.save('index.html')

Boston           20
Dorchester       15
DORCHESTER       13
Roxbury          10
Mattapan          7
Jamaica Plain     6
BOSTON            5
JAMAICA PLAIN     4
ROXBURY           4
MATTAPAN          3
Milton            3
Brookline         3
Roslindale        3
boston            3
Waban             3
Randolph          3
Hyde Park         3
South Boston      2
Winchester        2
BOSTON            2
EAST BOSTON       2
Vienna            2
HYDE PARK         2
SILVER SPRING     2
ROSLINDALE        1
Revere            1
CHARLESTOWN       1
BRIGHTON          1
DEDHAM            1
Andover           1
EVERETT           1
SHARON            1
FRANKLIN          1
Cumberland        1
East boston       1
ARLINGTON         1
MEDFORD           1
Squantum          1
Norwood           1
Tauton            1
STOUGHTON         1
Cambridge         1
WALTHAM           1
South Easton      1
Brockton          1
Allston           1
BRIGHTON          1
CANTON            1
roxbury           1
EAST BOSTON       1


In [52]:
#Use city and create table
import matplotlib.pyplot as plt
'''Okay to save the data returned by value_counts I am converting it to a df, and to save the df for each violation I am saving it to a dictionary'''
violations_by_city = {} #key: violation name, value: df that contains the location and frequency of the violations

# Go through each of the hazardous conditions violations
for name, code in violations.items():
    # Look at only this violation (code name in the dataset have other values beyond the code # so had to use contains)
    mask = df['code'].str.contains(code, case=False, na=False)
    df_code =  df[mask]
    # Count the number of violations for each address
    count = df_code['contact_city'].value_counts()
    # Convert value counts into a new dataframe to save the data
    df_violation = pd.DataFrame(count)
    df_violation = df_violation.reset_index()
    df_violation.columns = ['Location', 'Counts']
    violations_by_city[name] = df_violation
    